# Home assignment - quiz replacement
#### Daniel Levi - 315668129

## Part 1 - Implement basic unigram, bigram, and trigram language models

Loading packages

In [1]:
import itertools
import nltk
import re
import wget
from sys import getsizeof
import os
import random

from collections import defaultdict, Counter

nltk.download('punkt', quiet=True) # this module is used to tokenize the text

SEED = 1246
random.seed(SEED)

Some utilities to manipulate the corpus

In [2]:
# This code section was taken from lab2-1, and modified for my needs

def preprocess(text):
    """Strips #comments and empty lines from a string
    """
    result = []
    for line in text.split("\n"):
        line = line.strip()
        line = re.sub('#.*$', '', line)
        if line != '':
            result.append(line)
    return result

def nltk_normpunc_tokenize(str):
    return nltk.tokenize.word_tokenize(str.lower())

def split(list, portions, offset):
    return ([list[i] for i in range(0, len(list)) if i%portions != offset],
          [list[i] for i in range(0, len(list)) if i%portions == offset])

def SMSSpamCollection_tokenize(lines):
    result = []
    for line in lines:
        # tokenize
        tokens = nltk_normpunc_tokenize(line)
        if tokens[0] == "ham":
            tokens[0] = "HAM"
        elif tokens[0] == "spam":
            tokens[0] = "SPAM"
        # add a start of message token
        result += ["<s>"] + (tokens[:10] if len(tokens) >= 10 else tokens) + ["</s>"] # The truncation of the sentences(as written in the pdf report)

    return result

def postprocess(tokens):
    return ' '.join(tokens)\
                .replace("<s> ", "\n")

Download the corpus

In [3]:
corpus_filename = ("https://raw.githubusercontent.com/DanielLevi6/NLP-home-assignment/main/data/SMSSpamCollection.txt")
os.makedirs('data', exist_ok=True)
wget.download(corpus_filename, out="data/SMSSpamCollection.txt", bar=None)

'data/SMSSpamCollection (1).txt'

Load the corpus, read and tokenize it

In [4]:
with open("data/SMSSpamCollection.txt", 'r') as fin:
    lines = random.choices(preprocess(fin.read()), k=100)
    train_lines, test_lines = split(lines, 10, 0)
    train_tokens = SMSSpamCollection_tokenize(train_lines)
    test_tokens = SMSSpamCollection_tokenize(test_lines)

Extract vocabulary from dataset

In [5]:
vocabulary = list(set(train_tokens)) + list(set(test_tokens))

Generating the n-grams

In [6]:
# This code section was taken from lab2-1

# All possible n-grams
def all_ngrams(vocabulary, n):
    return list(itertools.product(vocabulary, repeat=n))

# The n-grams which appear in the text
def ngrams(tokens, n):
    return [tuple(tokens[i:i+n]) for i in range(0, len(tokens)-n+1)]

### Counting n-grams
Creates a 2-D dictionary of all the counts of the possible context+target token\
    - index : context, target token\
    - value : count of the context

In [7]:
# This code section was taken from lab2-1

def ngram_counts(vocabulary, tokens, n):
    context_dict = defaultdict(lambda: defaultdict(float))
    for context in all_ngrams(vocabulary, n-1):
        for target in vocabulary:
            context_dict[context][target] = 0.0

    for ngram, count in Counter(ngrams(tokens, n)).items():
        context_dict[ngram[:-1]][ngram[-1]] = count

    return context_dict

Creates the count dictionary for each of the model I want to implement

In [8]:
unigram_counts = ngram_counts(vocabulary, train_tokens, 1)
bigram_counts = ngram_counts(vocabulary, train_tokens, 2)
trigram_counts = ngram_counts(vocabulary, train_tokens, 3)

I want to count the size of each dictionary and get perspective about the size of our model

In [9]:
# This code section was taken from lab2-1

tokens_count = len(train_tokens)
unigram_count = sum(len(unigram_counts[cntxt]) for cntxt in unigram_counts)
bigram_count = sum(len(bigram_counts[cntxt]) for cntxt in bigram_counts)
trigram_count = sum(len(trigram_counts[cntxt]) for cntxt in trigram_counts)

# Report on the totals
print(f"Tokens: {tokens_count:6}\n"
     f"Unigrams: {unigram_count:6}\n"
     f"Bigrams: {bigram_count:6}\n"
     f"Trigrams: {trigram_count:6}\n")

Tokens:   1022
Unigrams:    424
Bigrams: 179776
Trigrams: 76225024



### The n-gram model implementation

In [10]:
def ngram_model(ngram_counts):
    prob_dict = defaultdict(lambda: defaultdict(float))
    for context in ngram_counts.keys():
        sigma = sum(ngram_counts[context].values())
        for target in ngram_counts[context].keys():
            if sigma == 0:
                prob_dict[context][target] = 0.0
            else:
                prob_dict[context][target] = (ngram_counts[context][target] / sigma)
            
    return prob_dict

Generating the actual desired models

In [11]:
unigram_model = ngram_model(unigram_counts)
bigram_model = ngram_model(bigram_counts)
trigram_model = ngram_model(trigram_counts)

The next cell was taken from lab2-1, for checking the size of the models that we created. That is crucial for our experiment to think about this issue(issue that forced me to reduce the amount of data I used for these models)

In [12]:
print(f"Tokens: {getsizeof(train_tokens):6}\n"
      f"Unigrams: {getsizeof(unigram_model):6}\n"
      f"Bigrams: {getsizeof(bigram_model):6}\n"
      f"Trigrams: {getsizeof(trigram_model):6}")

Tokens:   8440
Unigrams:    240
Bigrams:  18528
Trigrams: 10485864


We can see the typical words for spam messages, like-fantasy, free, urgent

In [13]:
for target, prob in trigram_model[("<s>", "SPAM")].items():
    if prob > 0:
        print(target)

congrats
hey
sunshine
please
urgent
u
someone
ur
sorry
free
last
fantasy


In [14]:
# This code was taken from lab2-1 for testing the models

def sample(model, context):
    distrib = model[context]
    prob_remaining = random.random()
    for token, prob in sorted(distrib.items()):
        if prob_remaining < prob:
            return token
        else:
            prob_remaining -= prob
    return ''

Generates 10 tokens context

In [15]:
def sample_sequence(model, start_context, count=10):
    random.seed(SEED)
    whole_context = list(start_context)
    for index in range(count):
        next_sample = sample(model, tuple(whole_context[index:]))
        whole_context.append(next_sample)
        
    return whole_context

I will test the 3 models for evaluating the context generating process

In [16]:
print(sample_sequence(unigram_model, ()))
print(sample_sequence(bigram_model, ("<s>",)))
print(sample_sequence(trigram_model, ("<s>", "SPAM",)))

['</s>', 'go', 'out', 'HAM', 'the', 'in', 'how', 'shall', '<s>', 'dun']
['<s>', 'HAM', 'no', 'money', '4', 'walsall', 'tue', '6', 'th', 'march', '</s>']
['<s>', 'SPAM', 'fantasy', 'football', 'is', 'back', 'on', 'your', 'tv', '.', 'go', '</s>']


### Perplexity calculation
Just as we did in lab2-1, I will implement the neglogprob calculation, for simplifying the perplexity calculation

In [17]:
import math

def neglogprob(tokens, model, n):
    score = 0.0
    context = tokens[0:n-1]
    for token in tokens[n-1:]:
        prob = model[tuple(context)][token]
        if(prob != 0):
            score += (-math.log2(prob))
        else:
            score += math.inf
        context = (context + [token])[1:]
    
    return score


def perplexity(tokens, model, n):
    nlog = neglogprob(tokens, model, n)
    N = (len(tokens) -n +1)
    return 2**(nlog / N)

print(f"Test perplexity - unigram: {perplexity(test_tokens, unigram_model, 1):.3f}\n"
      f"Test perplexity - bigram: {perplexity(test_tokens, bigram_model, 2):.3f}\n"
      f"Test perplexity - trigram: {perplexity(test_tokens, trigram_model, 3):.3f}\n")

Test perplexity - unigram: inf
Test perplexity - bigram: inf
Test perplexity - trigram: inf



## Part 2 - smoothing
### Add-delta smoothing

In [18]:
# Delta smoothing

def ngram_model_smoothed(ngram_counts, delta=2):
    prob_dict = defaultdict(lambda: defaultdict(int))
    for context in ngram_counts:
        V = len(ngram_counts[context])
        sigma = sum(ngram_counts[context].values())
        norm_factor = sigma + (delta * V)
        for target in ngram_counts[context].keys():
            prob_dict[context][target] = (ngram_counts[context][target] + delta / norm_factor)
            
    return prob_dict

In [19]:
trigram_model_smoothed = ngram_model_smoothed(trigram_counts, 1)
print(f"Test delta=1 smoothed perplexity - trigram: {perplexity(test_tokens, trigram_model_smoothed, 3):.3f}\n")

Test delta=1 smoothed perplexity - trigram: 70.476



In [20]:
trigram_model_smoothed = ngram_model_smoothed(trigram_counts, 2)
print(f"Test delta=2 smoothed perplexity - trigram: {perplexity(test_tokens, trigram_model_smoothed, 3):.3f}\n")

Test delta=2 smoothed perplexity - trigram: 70.204



### Knesser-Nay smoothing

In [21]:
def ngram_model_smoothed_kneser_nay(ngram_counts):
    prob_dict = defaultdict(lambda: defaultdict(float))
    d = 0.75
    for context in ngram_counts.keys():
        sigma = sum(ngram_counts[context].values())
        unique = 0
        for target in ngram_counts[context].keys():
            unique += 1
        lamda = d * (unique / sigma) if sigma > 0 else 0
        for target in ngram_counts[context].keys():
            P = unique / len(ngram_counts[context])
            prob_dict[context][target] = (abs(ngram_counts[context][target] - d) / (sigma + 1)) + (lamda*P)
            
    return prob_dict

In [22]:
trigram_model_kneser_ney_smoothed = ngram_model_smoothed_kneser_nay(trigram_counts)
print(f"Test kneser_nay smoothed perplexity - trigram: {perplexity(test_tokens, trigram_model_kneser_ney_smoothed, 3):.3f}\n")

Test kneser_nay smoothed perplexity - trigram: 0.275

